`Fama-French`因子是`empyrical`包计算的基础数据，用于投资组合风险和绩效分析。`empyrical`包的数据取材于美国市场，不适用A股投资组合分析。本文尝试使用`Pipeline`，基于[Fama-French三因子模型](https://www.bogleheads.org/wiki/Fama_and_French_three-factor_model)，参考论文作者提供的[构造方法](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html)每天定期运行计算出相应数据，修改`empyrical`包即可使用源于A股市场的因子数据，使得真正能用于A股投资组合分析。相关[代码](https://www.quantopian.com/posts/computing-the-fama-french-factors-with-pipeline-1)及[材料](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html)，主要来自于`quantopian`论坛。

quantopian最近发布的`pipeline`API可以快速运行大型股票的计算。这创造了一个广阔的世界，其中之一就是[Fama-French Three Factor Model](https://www.bogleheads.org/wiki/Fama_and_French_three-factor_model)。计算这些因子需要划分大量的股票，这些涉及数以千计的股票：在`Pipeline`之前，这在Quantopian平台上是不可能的。现在它变成了可能。

本文目的是生成`empyrical`包中`get_fama_french`函数所要获取的数据源，使其能用于A股投资分析。

## 原始5因子数据

### `quantopian/empyrical`数据

In [1]:
### 调用数据
import pandas as pd

In [2]:
fama_french = pd.read_pickle('fama_french_5.pkl')

In [3]:
fama_french.tail()

,Mkt-RF,SMB,HML,RF,Mom
Date,,,,,
2018-03-23 00:00:00+00:00,-0.0209,0.0021,-0.0019,0.00006,-0.0036
2018-03-26 00:00:00+00:00,0.0267,-0.0069,0.0002,0.00006,0.0132
2018-03-27 00:00:00+00:00,-0.0187,-0.0024,0.0040,0.00006,-0.0179
2018-03-28 00:00:00+00:00,-0.0035,0.0010,0.0064,0.00006,-0.0117
2018-03-29 00:00:00+00:00,0.0141,-0.0029,-0.0022,0.00006,0.0060


## 背景

### 三因子

<img src="images/median_me.gif">

<table>
<tr>
<td><img src="images/bh_small_v.gif" style="height:120px"></td>
<td><img src="images/bh_big_v.gif" style="height:120px"></td>
</tr>
<tr>
<td><img src="images/bh_small_n.gif" style="height:120px"></td>
<td><img src="images/bh_big_n.gif" style="height:120px"></td>
</tr>
<tr>
<td><img src="images/bh_small_g.gif" style="height:120px"></td>
<td><img src="images/bh_big_g.gif" style="height:120px"></td>
</tr>
</table>

据[因子构造方法](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html)：

+ Fama-French因子是由6个根据市值和账面价值划分，等权重资产组合构建形成的。请参阅6个市值/账面价值资产组合的描述。

+ SMB(Small Minus Big)为三个等权重小市值投资组合的平均回报减去三个大市值等权投资组合的平均回报。$$SMB = \frac{1}{3}*(Small Value + Small Neutral + Small Growth)
 - \frac{1}{3}*(Big Value + Big Neutral + Big Growth) $$

+ HML(High Minus Low)是两个等权市值组合的平均收益减去两个等权增长组合的平均收益。$$HML = \frac{1}{2}*(Small Value + Big Value)
 - \frac{1}{2}*(Small Growth + Big Growth)$$

+ `Rm-Rf`为市场超额收益，为股票收益率减去一个月国库券资金成本。

股票范围：
 + `Rm-Rf`包含所有股票
 + `SMB和HML`要求`t-1年`账面价值为正数；`t-1年12月`至`t年6月`有市值数据

### 五因子

五因子模型是在三因子的基础上加上了盈利因子`RMW`和投资因子`CMA`。

1. 盈利因子`RMW`：盈利好和盈利差的多元化股票组合收益之间的差异。其中盈利定义为年营业收入减去营业成本、利息费用、销售费用和管理费用后再除以t-1财年末的账面权益。

2. 投资因子CMA：投资高和投资低的多元化股票组合收益之间的差异。其中投资定义为 t-1财年的新增总资产除以t-2财年末的总资产 。

作者：小官大人
链接：https://www.zhihu.com/question/29558634/answer/257089971

[因子构造方法](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_5developed.html)：

+ 所有回报包括股息和资本收益，且不会持续复合计算。`Market`是一个地区的等权重股票组合的收益减去一个月的国库券利率。

`Fama/French 5 因子(2x3)`使用(1)市值和账面价值基础上构造的6个等权组合；(2)市值和营运盈利能力基础上构造的6个等权组合；(3)市值和投资基础上构造的6个等权组合等三项来构造。

为了构建`SMB`、`HML`、`RMW`和`CMA`因子，在每年六月底，我们将一个地区的股票分为两类市值和三个相应的账面市值(B/M)，运营盈利能力（OP）和投资（INV）组。大型股票指的是该地区6月份市值位于最高90％以上的股票，而小型股票指的是市值位于底部10％以下的股票。一个地区的B/M，OP和INV断点是该地区大型股票的相应比率的第30和第70百分位。

SMB(Small Minus Big)是九个小型股票投资组合的平均回报减去九个大股票投资组合的平均回报：$$SMB_{(B/M)} = \frac{1}{3} * (Small Value + Small Neutral + Small Growth) - \frac{1}{3} * (Big Value + Big Neutral + Big Growth)$$ 
$$SMB_{(OP)} = \frac{1}{3} * (Small Robust + Small Neutral + Small Weak) - \frac{1}{3} * (Big Robust + Big Neutral + Big Weak)$$ 
$$SMB_{(INV)} = \frac{1}{3} * (Small Conservative + Small Neutral + Small Aggressive) - \frac{1}{3} * (Big Conservative + Big Neutral + Big Aggressive)$$ 
$$SMB = \frac{1}{3} *  (SMB_{(B/M)} + SMB_{(OP)} + SMB_{(INV)})$$

`HML(High Minus Low)`是两个价值组合的平均收益减去两个增长组合的平均收益:

$$HML = \frac{1}{2} *  (Small Value + Big Value) - \frac{1}{2} *  (Small Growth + Big Growth)$$

`RMW(Robust Minus Weak)`是两个强劲的运营盈利能力组合的平均回报减去两个疲软的运营盈利能力组合的平均回报：$$RMW = \frac{1}{2} *  (Small Robust + Big Robust) - \frac{1}{2} *  (Small Weak + Big Weak)$$

`CMA(Conservative Minus Aggressive)`是两个保守投资组合（此处指选取公司投资风格保守的股票作为投资组合）的平均回报减去两个积极投资组合的平均回报：$$CMA = \frac{1}{2} *  (Small Conservative + Big Conservative) - \frac{1}{2} *  (Small Aggressive + Big Aggressive)$$

### 函数所需数据
函数`get_fama_french`中描述`five_factors`实际上并非上文说所说五因子，而是三因子加`MOM`组成。这点从`pyfolio`包中`rolling_regression`函数可以验证。实际上，该函数也仅仅使用了"SMB, HML, Mom"三列数据。

[每日动量因子的详细信息](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_mom_factor_daily.html)原文大致翻译：

我们使用在规模与(2-12)先前收益率基础上划分的六个价值加权组合来构建`Mom`。每天形成的投资组合是两个投资组合(market equity, ME)和3个投资组合（2-12）形成的投资组合的交集。每日规模的断点是纽约证券交易所的市场资产的中位数。每日先前（2-12）收益率断点是纽约证券交易所第30和第70个百分点。

`Mom`是两个高先前收益率组合的平均减去两个低先前收益率组合的平均回报之后的平均值。

+ SMB、HML同2.1节所述；
+ Mom定义如下：
$$Mom = \frac{1}{2} *  (Small High Prior + Big High Prior) - \frac{1}{2} *  (Small Low Prior + Big Low prior)$$

其中：
    1. `Big`意味着公司市值大于中位数，`Small`意味着公司市值低于中位数；
    2. `Prior return`计算`t-250`到`t-21`期间的收益率；
    3. `Low`意味着30%以下排位；`High`意味着70%以上排位。

用于构造`Mon`的六个投资组合包括纽约证券交易所，美国证券交易所和纳斯达克股票，股票必须有先前收益率数据。在第t天（在t-1日结束时形成）包含在投资组合中的股票必须具有t-251日结束时的价格和t-21的收益率。此外，从t-250至t-22的任何缺失收益率标记为-99.0，即CRSP的缺失价格代码。每个包含的股票在第t-1日结束时也必须有ME。

## 构造因子

### 常数

In [4]:
# 每月交易天数（近似值20，不同于美国股市，A股每年交易天数大约为244天）
normal_days = 31
business_days = int(0.66 * normal_days)

### 辅助函数

In [5]:
from zipline import get_calendar

calendar = get_calendar('SZSH')

all_trading_days = calendar.schedule.index
all_trading_days = all_trading_days[all_trading_days <= calendar.actual_last_session]
# 交易日期包含当天，但此时并没有当天的数据
#all_trading_days = all_trading_days[:-1]

/home/ldf/anaconda3/envs/zipline/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.fundamentals import Fundamentals

In [7]:
class Returns(CustomFactor):
    """
    每个交易日每个股票窗口长度"business_days"期间收益率
    """
    window_length = business_days
    inputs = [USEquityPricing.close]

    def compute(self, today, assets, out, price):
        out[:] = (price[-1] - price[0]) / price[0]


class PriorReturns(CustomFactor):
    """
    每个交易日每个股票窗口长度250的先前收益率
    
    p[-21] / p[-250] - 1
    """
    window_length = 250
    inputs = [USEquityPricing.close]

    def compute(self, today, assets, out, price):
        out[:] = (price[-21] - price[0]) / price[0]


class MarketEquity(CustomFactor):
    """
    每个交易日每只股票所对应的总市值(取期初数)
    """
    window_length = business_days
    inputs = [USEquityPricing.tmv]

    def compute(self, today, assets, out, mcap):
        out[:] = mcap[0]


class BookEquity(CustomFactor):
    """
    每个交易日每只股票所对应的账面价值（所有者权益）(取期初数)
    """
    window_length = business_days
    inputs = [Fundamentals.balance_sheet.A107]

    def compute(self, today, assets, out, book):
        out[:] = book[0]

以下需要关注：
 1. 是否将停牌的股票排除（停牌期间收益率为0，影响各组合的平均值。是否排除，取决于拟合优度。后续再进行验证。)
 2. 

In [8]:
def make_3_pipeline():
    """构造3因子pipeline"""
    mkt_cap = MarketEquity()
    book_equity = BookEquity()
    returns = Returns()
    be_me = book_equity / mkt_cap
    return Pipeline(
        columns={
            'market_cap': mkt_cap,
            'be_me': be_me,
            'returns': returns,
            'prior_returns': PriorReturns()
        })

In [9]:
from zipline.data.benchmarks_cn import get_cn_benchmark_returns
from zipline.data.treasuries_cn import get_treasury_data

In [10]:
def get_rm_rf(earliest_date, symbol='000300'):
    """
    Rm-Rf(市场收益 - 无风险收益)
    基准股票指数收益率 - 国库券1个月收益率
    
    输出pd.Series(日期为Index), 'Mkt-RF', 'RF'二元组
    """
    start = '1990-1-1'
    end = pd.Timestamp('today')
    benchmark_returns = get_cn_benchmark_returns(symbol).loc[earliest_date:]
    treasury_returns = get_treasury_data(start,end)['1month'][earliest_date:]
    # 补齐缺省值
    treasury_returns = treasury_returns.reindex(benchmark_returns.index, method='ffill')
    return benchmark_returns, treasury_returns

In [11]:
from zipline.research import run_pipeline

In [12]:
def compute_3_factors(one_day):
    """
    根据每天运行的pipeline结果，近似计算Fama-French 因子
    """
    factors = run_pipeline(make_3_pipeline(), one_day, one_day)
    # 获取后续使用的值
    returns = factors['returns']
    mkt_cap = factors.sort_values(['market_cap'], ascending=True)
    be_me = factors.sort_values(['be_me'], ascending=True)
    prior_returns = factors['prior_returns']

    # 根据市值划分总体，构造6个资产组合
    half = int(len(mkt_cap) * 0.5)
    small_caps = mkt_cap[:half]
    big_caps = mkt_cap[half:]

    thirty = int(len(be_me) * 0.3)
    seventy = int(len(be_me) * 0.7)
    growth = be_me[:thirty]
    neutral = be_me[thirty:seventy]
    value = be_me[seventy:]
    

    small_value = small_caps.index.intersection(value.index)
    small_neutral = small_caps.index.intersection(neutral.index)
    small_growth = small_caps.index.intersection(growth.index)

    big_value = big_caps.index.intersection(value.index)
    big_neutral = big_caps.index.intersection(neutral.index)
    big_growth = big_caps.index.intersection(growth.index)

    # 假设等权分配资金，取其投资组合回报率平均值
    sv = returns[small_value].mean()
    sn = returns[small_neutral].mean()
    sg = returns[small_growth].mean()

    bv = returns[big_value].mean()
    bn = returns[big_neutral].mean()
    bg = returns[big_growth].mean()
    
    # 按降序排列
    # 小市值组合收益率（序列没有列可以指定)
    s_p = prior_returns.loc[small_caps.index].sort_values(ascending=False)
    
    # 大市值组合收益率
    b_p = prior_returns.loc[big_caps.index].sort_values(ascending=False)
       
    shp = s_p.iloc[:int(len(s_p) * 0.3)].mean() # 小市值组合收益率前70%的均值
    bhp = b_p.iloc[:int(len(b_p) * 0.3)].mean() # 大市值组合收益率前70%的均值
    slp = s_p.iloc[int(len(s_p) * 0.7):].mean() # 小市值组合收益率后30%的均值
    blp = b_p.iloc[int(len(b_p) * 0.7):].mean() # 大市值组合收益率后30%的均值
       
    # 计算 SMB
    smb = (sv + sn + sg) / 3 - (bv + bn + bg) / 3

    # 计算 HML
    hml = (sv + bv) / 2 - (sg + bg) / 2
    
    # 计算 MOM
    mom = (shp + bhp) / 2 - (slp + blp) / 2

    return smb, hml, mom

In [13]:
def get_3_factors_data(dates, symbol):
    """计算指定日期列表"""
    start = dates[0]
    benchmark_returns, treasury_returns = get_rm_rf(start, symbol)
    res = []
    for d in dates:
        smb, hml, mom = compute_3_factors(d)
        mkt_rf = benchmark_returns.loc[d]
        rf = treasury_returns.loc[d]
        res.append((d, smb, hml, mom, mkt_rf, rf))
    df = pd.DataFrame.from_records(
        res, columns=['date', 'SMB', 'HML', 'Mom', 'Mkt-RF', 'RF'])
    df.set_index('date', inplace=True)
    return df

### 处理三因子数据

处理方式：
 1. 数据以'pkl'格式保存在本地文件，默认路径'~/stockdata/ff_factors/ff.pkl'
 2. 初始化时计算所有交易日的数据
 3. 以后每日刷新数据
 4. 修改`empyrical`包读取五因子部分，更改为从本地读取

In [14]:
from cswd.common.utils import data_root
import os
import pandas as pd

In [16]:
class FFFactor(object):
    # 沪深300指数自2002-01-07开始
    # 国库券数据1个月资金成本自2013-4-26开始
    # 数据最早开始日期设定为2013-5-1之后的最早交易日期
    earliest_date = pd.Timestamp('2013-05-01').tz_localize('UTC')

    def __init__(self, benchmark_symbol='000300', file_path=None):
        if file_path:
            self.file_path = file_path
        else:
            self.file_path = os.path.join(data_root('ff_factors'), 'ff.pkl')
        self.benchmark_symbol = benchmark_symbol

    def _read(self):
        try:
            return pd.read_pickle(self.file_path)
        except FileNotFoundError:
            return None

    def refresh(self):
        local_df = self._read()
        if local_df is None:
            dates = all_trading_days[all_trading_days > self.earliest_date]
            data = get_3_factors_data(dates, self.benchmark_symbol)
            data.to_pickle(self.file_path)
        else:
            last_date = local_df.index[-1]
            if last_date < all_trading_days[-1]:
                dates = all_trading_days[all_trading_days > last_date]
                data = get_3_factors_data(dates, self.benchmark_symbol)
                data = pd.concat([local_df, data])
                data.to_pickle(self.file_path)

    @property
    def data(self):
        print('正在整理三因子数据，请耐心等待......')
        self.refresh()
        return self._read()

In [17]:
ff = FFFactor()
df = ff.data

正在整理三因子数据，请耐心等待......
